<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/computer-vision/2_Cats_vs_Dogs_Kaggle_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download your Kaggle API keys to local system and then upload to colab environment

In [ ]:
from google.colab import files
files.upload()

In [2]:
!mkdir ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
# Change access to your Kaggle API so only current account has access to read/write
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c dogs-vs-cats

 98% 792M/812M [00:03<00:00, 219MB/s]
100% 812M/812M [00:03<00:00, 222MB/s]


In [6]:
!unzip --qq dogs-vs-cats.zip

In [7]:
!unzip --qq train.zip

In [8]:
!unzip --qq test1.zip

In [9]:
import os, shutil, pathlib

In [10]:
original_dir = pathlib.Path("train")
new_dir = pathlib.Path("cats_vs_dogs")

In [11]:
def create_folder_structure(name, start_index, end_index):

  for category in ("cat", "dog"):

    dir=new_dir / name / category
    os.makedirs(dir)

    file_names = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]

    for files in file_names:
      shutil.copyfile(src=original_dir / files, dst=dir / files)

In [12]:
create_folder_structure("train", start_index=0,end_index = 1000)
create_folder_structure("validation", start_index=1000, end_index=1500)
create_folder_structure("test", start_index=1500, end_index=2500 )

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
def getmodel():

  inputs = keras.Input(shape=(180, 180, 3))

  #rescale the image to 1 - 255
  x = layers.Rescaling(scale=1./255)(inputs)

  x = layers.Conv2D(filters=32, activation="relu", kernel_size=3) (x)
  x = layers.MaxPool2D(pool_size=(2,2))(x)

  x = layers.Conv2D(filters=64, activation="relu", kernel_size=3) (x)
  x = layers.MaxPool2D(pool_size=(2,2))(x)

  x = layers.Conv2D(filters=128, activation="relu", kernel_size=3) (x)
  x = layers.MaxPool2D(pool_size=(2,2))(x)

  x = layers.Conv2D(filters=256, activation="relu", kernel_size=3) (x)
  x = layers.MaxPool2D(pool_size=(2,2))(x)

  x = layers.Conv2D(filters=256, activation="relu", kernel_size=3) (x)
  
  x = layers.Flatten()(x)

  outputs = layers.Dense(1, activation="sigmoid")(x)

  model = keras.Model(inputs, outputs)

  model.compile(optimizer="rmsprop",
                loss="binary_crossentropy",
                metrics = ["accuracy"])
  
  return model

In [15]:
model = getmodel()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 180, 180, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 89, 89, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 43, 43, 64)       0         
 2D)                                                         

In [17]:
from tensorflow.keras.utils import image_dataset_from_directory

In [18]:
train_dataset = image_dataset_from_directory(new_dir / "train", batch_size=32, image_size=(180, 180))

Found 2000 files belonging to 2 classes.


In [19]:
validation_dataset = image_dataset_from_directory(new_dir / "validation", batch_size=32, image_size=(180, 180))

Found 1000 files belonging to 2 classes.


In [20]:
test_dataset = image_dataset_from_directory(new_dir / "test", batch_size=32, image_size=(180, 180))

Found 2000 files belonging to 2 classes.
